# Performing speech-to-text on files

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path
from tqdm.auto import tqdm
import pandas as pd
from IPython.display import display
import numpy as np

import speech_recognition as sr

In [6]:
# configure user
user = "mitchel"
extracted_feats = True

if user.lower() == "jonas":
    BASE_PATH = Path("/users/jonvdrdo/jonas/data/aaa_contextaware/raw/uz_study/")
elif user.lower() == "mitchel":
#     BASE_PATH = Path("Z:/shares/ghep_lab/2021_VanhollebekeKappen_EEGStudy2_MIST_Cyberball_Audio/")
    BASE_PATH = Path("D:/Data/EEG_Study_2")
DATA_PATH = BASE_PATH.joinpath("Data/Raw/Audio")

In [9]:
# https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python
filename = "D:/Data/EEG_Study_2/Data/Raw/Audio/ppt_8_mist_speech/audio_referential_control.wav"

# initialize the recognizer
r = sr.Recognizer()

# open the file
with sr.AudioFile(filename) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data, language="nl-NL")
    print(text)

op de afbeeldingen afbeelding van een rekensom x door 72 keer 2 en dan 30 eten komen en dan denk ik dat ik eerst die andere twee samen heb genomen tot dat 31 uitkwam en dan kan ik zo even uit denk dat ik wel redelijk op mijn gemak was was het toch erg dat je snel snel denken maar wordt in ieder geval heerlijk rustig en de regels opgesteld dat het 72/2 met een breuk met een scherm wordt weergegeven te haakjes en dan datum 27 en 4 en dan lekker tegen en scherpe vraagteken


In [11]:
# https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python
filename = "D:/Data/EEG_Study_2/Data/Raw/Audio/ppt_12_mist_speech/audio_referential_control.wav"

# initialize the recognizer
r = sr.Recognizer()

# open the file
with sr.AudioFile(filename) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data, language="nl-BE")
    print(text)

Ik zie op het scherm en rekenopgaven staan die staat op het bovenste deel van het scherm aan de rest van het scherm is wit de rekenopgaven omvat een deling eerst en vooral die tussen haakjes staat 72/2 daarna en daarna 2 aftrekkingen en met 27 en -4 sterren is gelijk aan teken en een vraagteken waarde het antwoord zou moeten komen de rekenopgaven is een vrij moeilijke rekenopgave aan moeilijker dan ik verwacht had om hier te zien te krijgen


In [24]:
# https://textblob.readthedocs.io/en/dev/

from textblob import TextBlob
import nltk
nltk.download('punkt') # Used for tags
nltk.download('averaged_perceptron_tagger') # Used for tags
nltk.download('brown') # Used for noun phrases

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mitch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mitch\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\mitch\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.


True

In [34]:
blob = TextBlob(text)

blob.tags

blob.noun_phrases

for sentence in blob.sentences:
    print(sentence.sentiment)

Sentiment(polarity=0.0, subjectivity=0.0)


In [44]:
# Now Dutch
# https://github.com/gvisniuc/textblob-nl

from textblob_nl import PatternTagger, PatternAnalyzer

blob = TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

# The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
blob.sentiment # Polarity, subjectivity

(-0.06333333333333334, 0.4766666666666667)

In [43]:
# Word count
blob.word_counts['ik'] # For instance 'ik'

# Relative word usage (proportional):
blob.word_counts['ik']  / len(blob.words)

0.024691358024691357

In [40]:
len(blob.words)

81